Load python packages

In [2]:
# I'm sure loads of these are unnecessary!

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd

In [3]:
parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir = os.path.join(parent_dir, 'Output/sat_plot')

#Products come from satellite_data_download.ipynb
chla_data = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_CHLA_April-May2024.nc'))
current_data = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_SSH_April-May2024.nc'))

#Chla colorbar 
color = 'YlGnBu_r'

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

In [21]:
last_date = chla_data.time[-2:]
# Filter the dataset to only include the last date
chla_data = chla_data.sel(time=last_date)

last_date = current_data.time[-2:]
# Filter the dataset to only include the last date
current_data = current_data.sel(time=last_date)


In [15]:
display(chla_data)

<xarray.Dataset> Size: 736MB
Dimensions:              (time: 2, latitude: 1056, longitude: 2880)
Coordinates:
  * latitude             (latitude) float32 4kB 55.01 55.02 ... 65.98 65.99
  * longitude            (longitude) float32 12kB -34.99 -34.98 ... -5.005
  * time                 (time) datetime64[ns] 16B 2024-05-20 2024-05-21
Data variables: (12/21)
    CHL                  (time, latitude, longitude) float32 24MB ...
    CHL_uncertainty      (time, latitude, longitude) float64 49MB ...
    DIATO                (time, latitude, longitude) float32 24MB ...
    DIATO_uncertainty    (time, latitude, longitude) float64 49MB ...
    DINO                 (time, latitude, longitude) float32 24MB ...
    DINO_uncertainty     (time, latitude, longitude) float64 49MB ...
    ...                   ...
    PICO_uncertainty     (time, latitude, longitude) float64 49MB ...
    PROCHLO              (time, latitude, longitude) float32 24MB ...
    PROCHLO_uncertainty  (time, latitude, longitude) float64 49MB ...
    PROKAR               (time, latitude, longitude) float32 24MB ...
    PROKAR_uncertainty   (time, latitude, longitude) float64 49MB ...
    flags                (time, latitude, longitude) int8 6MB ...
Attributes: (12/92)
    Conventions:                     CF-1.8, ACDD-1.3
    DPM_reference:                   GC-UD-ACRI-PUG
    IODD_reference:                  GC-UD-ACRI-PUG
    acknowledgement:                 The Licensees will ensure that original ...
    citation:                        The Licensees will ensure that original ...
    cmems_product_id:                OCEANCOLOUR_ATL_BGC_L3_NRT_009_111
    ...                              ...
    time_coverage_resolution:        P1D
    time_coverage_start:             2023-10-15T06:57:27Z
    title:                           cmems_obs-oc_atl_bgc-plankton_nrt_l3-mul...
    westernmost_longitude:           -46.0
    westernmost_valid_longitude:     -46.0
    copernicusmarine_version:        1.2.2

In [4]:
wmo_list = [4903532, 1902637]
#Float 1 = test float in the Icelandic Bassin
float_1_url = 'https://data-argo.ifremer.fr/dac/aoml/4903532/4903532_Sprof.nc'
#Float 2 = test float on Custard with glider next to it
float_2_url = 'https://data-argo.ifremer.fr/dac/coriolis/1902637/1902637_Sprof.nc'

#List the floats
floats_url = [float_1_url, float_2_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir =  os.path.join(parent_dir, 'Data/Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

In [5]:
#We will loop through the float files, we create two dataframe. One with all the positions from both floats, and one with only the last positions from both floats.
#Since we know plot the last position of the float at the date of the plot, the second one is not needed.

#Create empty df
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
last_position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})

for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    last_temp_df = temp_df[temp_df['JULD'] == max(temp_df['JULD'])]

    last_position_df = pd.concat([last_position_df, last_temp_df], ignore_index=True)
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()
position_df = position_df[position_df['LATITUDE'] > min_lat]

C:\Users\flapet\AppData\Local\Temp\ipykernel_10244\1791067301.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  last_position_df = pd.concat([last_position_df, last_temp_df], ignore_index=True)
C:\Users\flapet\AppData\Local\Temp\ipykernel_10244\1791067301.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_df = pd.concat([position_df, temp_df], ignore_index=True)


In [6]:
position_df.head()

,PROF_NUM,LONGITUDE,LATITUDE,float,JULD
0,1.0,-19.8661,59.9345,4903532,2023-05-07 07:30:52.000794624
1,2.0,-19.6498,60.4443,4903532,2023-05-17 04:56:51.000510464
2,3.0,-20.1017,60.4921,4903532,2023-05-27 02:10:14.000229376
3,4.0,-19.8409,60.7486,4903532,2023-06-05 23:48:13.002537472
4,5.0,-19.2193,61.0747,4903532,2023-06-15 20:47:59.002213888


In [18]:
# We load shape files for topographic lines
first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

In [25]:
test = xr.open_dataset(os.path.join(satellite_dir, 'neodass_chla_1day.nc'))

In [33]:
#Format the date of the loop as a string and use it to set the plot title and filename
date_of_plot = test['time'].data[0]
strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
plot_title = 'Chla concentration ' + strdate_of_plot
plotname = chla_plot_dir + '/' + 'Chla_' + strdate_of_plot + '.png'

#Filter the float position data to highlight the last position at the date of the plot
last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

#Set up the plot layout, extent and title
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
ax.set_title(plot_title,fontsize=24)

#Plot the chla colours and the land coastline
im = ax.pcolormesh(test['longitude'].data, test['latitude'].data, test['CHL_OC4ME'].data[0,:], cmap = color, norm=colors.LogNorm(), transform=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)

#Plot the float position
sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

#Plot the topo line of 1000 and 2000m depth
gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

#add the lon and lat grid 
gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
gl.top_labels=False   # suppress top labels
gl.right_labels=False # suppress right labels

#format the color bar
cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
cbar.set_label('Chla', rotation=270, labelpad=15)


#save the plot and then close it to avoid high memory usage
plt.savefig(plotname)
plt.clf()
plt.close()

In [34]:
max(test['CHL_OC4ME'].data[0,:])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [32]:
# define the caracteristics of the map and the features to plot
plotting_chla = True
if plotting_chla == True :
    chla_plot_dir = os.path.join(output_dir, 'Chla')
    if os.path.exists(chla_plot_dir):
        print(f"Chla plots saving...")
    else:
        os.makedirs(chla_plot_dir)
        print(f"Chla plots saving...")
    for i in tqdm(range(len(chla_data['time'].data))):

        #Format the date of the loop as a string and use it to set the plot title and filename
        date_of_plot = chla_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Chla concentration ' + strdate_of_plot
        plotname = chla_plot_dir + '/' + 'Chla_' + strdate_of_plot + '.png'

        #Filter the float position data to highlight the last position at the date of the plot
        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
        ax.set_title(plot_title,fontsize=24)

        #Plot the chla colours and the land coastline
        im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data, chla_data['CHL'].data[i,:],cmap = color, norm=colors.LogNorm(), transform=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE)

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
        cbar.set_label('Chla', rotation=270, labelpad=15)

        
        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()

Chla plots saving...


100%|██████████| 2/2 [01:48<00:00, 54.24s/it]


In [20]:
# define the caracteristics of the map and the features to plot
plotting_log = True
if plotting_log == True :
    chla_log_plot_dir = os.path.join(output_dir, 'Chla_log')
    if os.path.exists(chla_log_plot_dir):
        print(f"Chla log plots saving...")
    else:
        os.makedirs(chla_log_plot_dir)
        print(f"Chla log plots saving...")
    for i in tqdm(range(len(chla_data['time'].data))):

        #Format the date of the loop as a string and use it to set the plot title and filename
        date_of_plot = chla_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Chla concentration ' + strdate_of_plot
        plotname = chla_log_plot_dir + '/' + 'Chla_log_' + strdate_of_plot + '.png'

        #Filter the float position data to highlight the last position at the date of the plot
        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_title(plot_title,fontsize=24)
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())

        #Plot the chla colours and the land coastline
        im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,chla_data['CHL'].data[i,:],cmap = color, norm=colors.LogNorm(), transform=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE)

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
        cbar.set_label('Chla', rotation=270, labelpad=15)

        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()

Chla log plots saving...


100%|██████████| 2/2 [01:08<00:00, 34.25s/it]


In [22]:
plot_current = True
min_lon = -30 #for the ssh data the extent is lower
if plot_current == True:

    current_plot_dir = os.path.join(output_dir, 'Surface current')
    if os.path.exists(current_plot_dir):
        print(f"Surface current plots saving...")
    else:
        os.makedirs(current_plot_dir)
        print(f"Surface current plots saving...")

    #extract the lon and lat from the dataset only once
    x = current_data['longitude'].data
    y = current_data['latitude'].data

    for i in tqdm(range(len(current_data['time'].data))):

        #From the U and V vector compute the speed, we use it as our colour map
        u = current_data['ugos'].data[i,:]
        v = current_data['vgos'].data[i,:]
        speed = np.sqrt(u**2 + v**2)

        #can be directly added to the quiver function
        vector_crs = ccrs.PlateCarree()

        #Format the date of the loop as a string and use it to make the title and the filename
        date_of_plot = current_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Surface current ' + strdate_of_plot
        plotname = current_plot_dir + '/' + 'SSC' + strdate_of_plot + '.png'

        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(15, 15))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
        ax.set_title(plot_title,fontsize=24)

        #Plot the current vectors field and the coastline
        im = ax.quiver(x, y, u, v, speed, angles='xy', scale_units='xy', cmap='viridis', transform=vector_crs, regrid_shape=50, width = 0.002)
        ax.coastlines('50m')

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3)
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4)

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, label=r'Sea surface current (m s$^{-1}$)')
        cbar.set_label(r'Sea surface current (m s$^{-1}$)', rotation=270, labelpad=15)

        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()

Surface current plots saving...


100%|██████████| 2/2 [01:45<00:00, 52.98s/it]
